In [2]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%load_ext line_profiler
%load_ext memory_profiler

In [27]:
fc.eightb.preselection.ranked_quadh_lowm.signal_list

['/eos/uscms/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/ranked_quadh_lowm/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_700_MY_300_accstudies.root',
 '/eos/uscms/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/ranked_quadh_lowm/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_800_MY_300_accstudies.root',
 '/eos/uscms/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/ranked_quadh_lowm/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_800_MY_350_accstudies.root',
 '/eos/uscms/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/ranked_quadh_lowm/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_900_MY_300_accstudies.root',
 '/eos/uscms/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/ranked_quadh_lowm/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_900_MY_400_accstudies.root',
 '/eos/uscms/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/ranked_quadh_lowm/NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1000_MY_300_accstudies.root',
 '/eos/uscms/store/user/ekoenig/8BAnalysis/NTuples/2018/p

In [6]:
# %%memit
bkg = Tree(fc.eightb.preselection_ranked_quadh_multim.Bkg_MC_List)

100%|██████████| 17/17 [00:05<00:00,  2.95it/s]


In [6]:
'LHEPdfWeight_var33' in bkg.fields

False

In [ ]:
[f for f in bkg.fields if f == 'LHEPdfWeight_var33']

['LHEPdfWeight_var33']

['jet_E',
 'LHEPdfWeight_var25',
 'LHEPdfWeight_var17',
 'jet_btag',
 'LHEPdfWeight_var26',
 'H1Y1_score',
 'mu_1_eta',
 'H2Y2_pt',
 'H1Y1_b1_phi',
 'H2Y1_b2_ptRegressed',
 'PSWeight_var3',
 'H2Y1_phi',
 'H2Y1_b2_mRegressed',
 'H2Y2_b2_btag',
 'n_genjet',
 'PUWeight_up',
 'mu_1_m',
 'H1Y2_b1_ptRegressed',
 'H1Y1_b2_m',
 'dijet_dr',
 'H1Y2_b2_pt',
 'H1Y2_b2_mRegressed',
 'PSWeight_var0',
 'LHEPdfWeight_var2',
 'H2Y1_b2_eta',
 'ele_2_eta',
 'genjet_m',
 'dijet_j1Idx',
 'H2Y1_b1_btag',
 'LHEPdfWeight_var10',
 'LHEScaleWeight_var8',
 'jet_puid',
 'Run',
 'H1Y1_b2_pt',
 'H2Y1_eta',
 'H1Y2_b2_eta',
 'H2Y2_b2_ptRegressed',
 'dijet_phi',
 'jet_m',
 'ele_2_phi',
 'H2Y1_b2_score',
 'LHEPdfWeight_var32',
 'H1Y1_phi',
 'H1Y1_b2_mRegressed',
 'aplanarity',
 'b_6j_score',
 'jet_phi',
 'n_true_int',
 'genjet_phi',
 'X_pt',
 'LHEPdfWeight_var18',
 'H1Y2_b2_m',
 'LHEPdfWeight_var20',
 'LHEPdfWeight_var7',
 'H1Y1_pt',
 'LHEPdfWeight_var16',
 'jet_eta',
 'Y2_eta',
 'rhofastjet_all',
 'H1Y1_eta',
 'H1Y2_e